<a href="https://colab.research.google.com/github/hfathie/qso/blob/master/GPU_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-wpzchx8t
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-wpzchx8t
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4307 sha256=26934d1faa9fb4ec23d6d4209b20c4922fe8201e4f73b03c8ff17b5340076455
  Stored in directory: /tmp/pip-ephem-wheel-cache-g4yactkr/wheels/ca/33/8d/3c86eb85e97d2b6169d95c6e8f2c297fdec60db6e84cb56f5e
Successfully built NVCCPlugin


In [4]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [5]:
%%cu
#include <iostream>
    int
    main()
{
    std::cout << "Welcome To GeeksforGeeks\n";
    return 0;
}

Welcome To GeeksforGeeks



In [6]:
%%cu
#include <cstdio>
#include <iostream>
using namespace std;

__global__ void cuda_hello(){
  int a = 0;
}

int main(){
  cuda_hello<<<1, 1>>>();
  cout << "aaaaa";
  return 0;
}

aaaaa


In [10]:
%%cu
#include <cstdio>
#include <iostream>

	using namespace std;

__global__ void maxi(int* a, int* b, int n)
{
	int block = 256 * blockIdx.x;
	int max = 0;

	for (int i = block; i < min(256 + block, n); i++) {

		if (max < a[i]) {
			max = a[i];
		}
	}
	b[blockIdx.x] = max;
}

int main()
{

	int n;
	n = 3 >> 2;
	int a[n];

	for (int i = 0; i < n; i++) {
		a[i] = rand() % n;
		cout << a[i] << "\t";
	}

	cudaEvent_t start, end;
	int *ad, *bd;
	int size = n * sizeof(int);
	cudaMalloc(&ad, size);
	cudaMemcpy(ad, a, size, cudaMemcpyHostToDevice);
	int grids = ceil(n * 1.0f / 256.0f);
	cudaMalloc(&bd, grids * sizeof(int));

	dim3 grid(grids, 1);
	dim3 block(1, 1);

	cudaEventCreate(&start);
	cudaEventCreate(&end);
	cudaEventRecord(start);

	while (n > 1) {
		maxi<<<grids, block>>>(ad, bd, n);
		n = ceil(n * 1.0f / 256.0f);
		cudaMemcpy(ad, bd, n * sizeof(int), cudaMemcpyDeviceToDevice);
	}

	cudaEventRecord(end);
	cudaEventSynchronize(end);

	float time = 0;
	cudaEventElapsedTime(&time, start, end);

	int ans[2];
	cudaMemcpy(ans, ad, 4, cudaMemcpyDeviceToHost);

	cout << "The maximum element is : " << ans[0] << endl;

	cout << "The time required : ";
	cout << time << endl;
}


The maximum element is : 1445357584
The time required : 0.002976



In [2]:
%%writefile hgpu_test1.cu
#include <iostream>
using namespace std;


__global__ void add(int n, float *a, float *b){
    for(int i = 0; i < n; i++){
        b[i] = a[i] + b[i];
    }
}

int main(void){
    
    int N = 1 << 20;
    float *x, *y;

    // Allocate Unified Memory - Accessible from CPU and GPU.
    cudaMallocManaged(&x, N*sizeof(float));
    cudaMallocManaged(&y, N*sizeof(float));

    // Initiallize x and y arrays on the Host
    for(int i = 0; i < N; i++){
        x[i] = 1.0f;
        y[i] = 2.0f;
    }

    // Run kernel on 1M elements on GPU
    add<<<1, 1>>>(N, x, y);

    // Wait for GPU to finish before accessing on host
    cudaDeviceSynchronize();

    for(int i = 0; i < 5; i++){
        cout << x[i] << ", " << y[i] << endl;
    }

    // Free memory
    cudaFree(x);
    cudaFree(y);

    return 0;

}



Overwriting hgpu_test1.cu


In [24]:
!ls

hgpu_test1.cpp	hgpu_test1.cu  sample_data  src


In [26]:
%%shell
nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [27]:
%%shell
nvcc hgpu_test1.cu -o out1

In [28]:
%%shell
./out1

1, 3
1, 3
1, 3
1, 3
1, 3


In [31]:
%%shell
nvprof ./out1

==960== NVPROF is profiling process 960, command: ./out1
1, 3
1, 3
1, 3
1, 3
1, 3
==960== Profiling application: ./out1
==960== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  108.91ms         1  108.91ms  108.91ms  108.91ms  add(int, float*, float*)
      API calls:   67.99%  234.06ms         2  117.03ms  55.940us  234.00ms  cudaMallocManaged
                   31.64%  108.93ms         1  108.93ms  108.93ms  108.93ms  cudaDeviceSynchronize
                    0.18%  632.64us         2  316.32us  251.20us  381.44us  cudaFree
                    0.12%  408.14us         1  408.14us  408.14us  408.14us  cuDeviceTotalMem
                    0.04%  154.66us       101  1.5310us     142ns  64.631us  cuDeviceGetAttribute
                    0.02%  53.315us         1  53.315us  53.315us  53.315us  cudaLaunchKernel
                    0.01%  28.168us         1  28.168us  28.168us  28.168us  cuDeviceGetName
           

In [9]:
%%writefile hgpu_test1.cu
#include <iostream>
using namespace std;


__global__ void add(int n, float *a, float *b){
    int index = threadIdx.x;
    int stride = blockIdx.x;
    for(int i = index; i < n; i += stride){
      b[i] = a[i] + b[i];
    }
}

int main(void){
    
    int N = 1 << 20;
    float *x, *y;

    // Allocate Unified Memory - Accessible from CPU and GPU.
    cudaMallocManaged(&x, N*sizeof(float));
    cudaMallocManaged(&y, N*sizeof(float));

    // Initiallize x and y arrays on the Host
    for(int i = 0; i < N; i++){
        x[i] = 1.0f;
        y[i] = 2.0f;
    }

    // Run kernel on 1M elements on GPU
    add<<<1, 256>>>(N, x, y);

    // Wait for GPU to finish before accessing on host
    cudaDeviceSynchronize();

    for(int i = 0; i < 5; i++){
        cout << x[i] << ", " << y[i] << endl;
    }

    // Free memory
    cudaFree(x);
    cudaFree(y);

    return 0;

}



Overwriting hgpu_test1.cu


In [10]:
!ls

hgpu_test1.cu  outx2  sample_data


In [11]:
%%shell
nvcc hgpu_test1.cu -o outx2

In [ ]:
%%shell
./outx2